In [6]:
import numpy as np
import time
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import scipy.linalg as la

In [3]:
%%R
library(tidyverse)


UsageError: Cell magic `%%R` not found.


In [173]:
# Load Data
## Simulating Data
n = 1000
r = .95
inter = np.ones(n)
x1 = np.random.normal(0,5,n)
x2 = np.random.normal(0,2,n)
x34 = np.random.multivariate_normal([0, 0], np.array([[3, r], [r, 2]]), n)
X = np.c_[x1,x2,x34]
X = (X - X.mean(axis = 0)) / X.std(axis = 0)
X = np.c_[inter,X]

beta = np.array([1,-2,3,-1,2]).reshape(5,1)
pr = 1 / (1 + np.exp(X.dot(beta)))


# Possibly need to standardize this
y = np.random.binomial(n=1, p=pr)

In [32]:
np.c_[pr,y][:10,:]


array([[0.99541296, 1.        ],
       [0.55931515, 0.        ],
       [0.99031203, 1.        ],
       [0.99095291, 1.        ],
       [0.64595933, 1.        ],
       [0.11408075, 0.        ],
       [0.06377019, 0.        ],
       [0.94527421, 1.        ],
       [0.82094257, 1.        ],
       [0.81209617, 1.        ]])

In [27]:
n,d = X.shape

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
def log_normal_pdf(x,w,alpha):
    return(np.exp(-(np.log(x) - w)**2 / (2 * alpha**2)) / (x * alpha * np.sqrt(2 * np.pi)))

In [ ]:
def matlab_lognorm(x,w,alpha):
    return np.sum(-(.5*np.log(2*np.pi*alpha)) - ((x - w)**2) / (2*alpha))

In [ ]:
def logistic_rmhmc(y,X,niter,burnin,leap_frog_steps= 6,num_newton_steps = 6):
    n,d = X.shape
    step_size = 3/leap_frog_steps
    w = np.ones(d)*(1e-3)
    wSaved = np.zeros(niter - burnin,d)
    # Set up a dictionary for the gradients
    g_deriv = {}
    invGdG = {}
    trInvGdG = np.zeros(d)
    
    log_prior = sum(np.zeros(d),w,alpha)
    f = X.dot(log_prior)
    loglike = f.T.dot(y) - np.sum(np.log(1 + np.exp(f)))
    current = loglike + log_prior
    prop = 0
    
    for i in range(niter):
        wNew = w
        prop += 1
        
        # New G
        f = X.dot(wNew)
        p = 1 / (1+ np.exp(-f))
        v = p.dot(1 - p)
        G = X.T.dot(v.dot(X)) + np.identity(d)/alpha
        
        invG = la.inv(G)
        
        oG = G
        o_chol_G = la.cholesky(G)
        o_inv_G = invG
        
        for j in range(d):
            Z = ((1 - 2*p).dot(X[:,d])) # Review
            g_deriv[j] = X.T.dot(v.dot(np.diag(Z))).dot(X)
            invGdG[j] = InvG.dot(g_deriv[j])
            trInvGdG[j] = np.trace(invGdG[j])
            
        p_momentum = np.random.randn(p).dot(oG)
        o_momentum = p_momentum
        
        if np.random.randn(1) > .5:
            TimeStep = 1
        else:
            TimeStep = -1
            
        RandomSteps = np.ceil(np.random.rand(1)*leap_frog_steps)
        
        SavedSteps = wNew
        
        # Leapfrog steps
        for step in range(RandomSteps):
            
            # Updating the Momentum
            dLdTheta = (X.T.dot(y - (1/(1 + np.exp(-f))).reshape(y.shape))) - np.identity(d)*(1 / alpha).dot(wNew)
            trace1 = trInvGdG/2
            
            # Multiple fixed point iteration
            pm = p_momentum
            for fixed_i in range(num_newton_steps):
                Momentum

In [51]:
np.random.randn(3, 4)

array([ 0.51301974, -1.93166497,  0.53002304])

In [50]:
test_d = {}
test_d[0] = np.array([1,2,3,4,5,6]).reshape(2,3)
test_d

{0: array([[1, 2, 3],
        [4, 5, 6]])}

In [45]:
np.diag([1,2,3,4,5])

array([[1, 0, 0, 0, 0],
       [0, 2, 0, 0, 0],
       [0, 0, 3, 0, 0],
       [0, 0, 0, 4, 0],
       [0, 0, 0, 0, 5]])

In [36]:
test_x = np.random.multivariate_normal([0, 0], np.array([[3, r], [r, 2]]),2)
test_x
la.inv(test_x).dot(test_x)

array([[1.00000000e+00, 3.20499729e-17],
       [1.70818855e-16, 1.00000000e+00]])

In [43]:
A = np.array([
    [1,3,5],
    [3,13,23],
    [5,23,42]
])
print(la.cholesky(A))
print(la.inv(A))
print(la.inv(A).dot(A))

[[1. 3. 5.]
 [0. 2. 4.]
 [0. 0. 1.]]
[[ 4.25 -2.75  1.  ]
 [-2.75  4.25 -2.  ]
 [ 1.   -2.    1.  ]]
[[ 1.00000000e+00  3.77475828e-15  1.33226763e-15]
 [-4.44089210e-16  1.00000000e+00  1.33226763e-14]
 [ 9.99200722e-16 -5.55111512e-16  1.00000000e+00]]


In [28]:
X.std(axis = 0)

array([nan,  1.,  1.,  1.,  1.])

In [327]:
y = y
X = X
niter = 10
burnin = 2
leap_frog_steps = 6
num_newton_steps = 6
alpha = 100


n,d = X.shape
step_size = 3/leap_frog_steps
w = np.ones((d,1))*(1e-3)
wSaved = np.zeros((niter - burnin,d))
# Set up a dictionary for the gradients
g_deriv = {}
invGdG = {}
trInvGdG = np.zeros(d)
    
log_prior = matlab_lognorm(np.zeros(d),w,alpha)
f = X.dot(log_prior)
loglike = f.T.dot(y) - np.sum(np.log(1 + np.exp(f)))
currentLJL = loglike + log_prior
prop = 0
acc = 0
    
for i in range(niter):
    wNew = w
    prop += 1
        
        # New G
    f = X.dot(wNew)
    p = 1 / (1+ np.exp(-f))
    v = p*(1 - p)
    G = X.T.dot(v*(X)) + np.identity(d)/alpha
        
    invG = la.inv(G)
        
    oG = G
    o_chol_G = la.cholesky(G)
    o_inv_G = invG
        
    for j in range(d):
        Z = ((1 - 2*p)*(X[:,j])) # Review
        g_deriv[j] = X.T.dot(v*(np.diag(Z))).dot(X) # My v is a scalar, but should it be?
        invGdG[j] = invG.dot(g_deriv[j])
        trInvGdG[j] = np.trace(invGdG[j])
        if j == d-1:
            print("j= ", j )
            
    p_momentum = np.random.randn(d).dot(oG)
    print(p_momentum)
    o_momentum = p_momentum
        
    if np.random.randn(1) > .5:
        TimeStep = 1
    else:
        TimeStep = -1
            
    RandomSteps = np.ceil(np.random.rand(1).item()*leap_frog_steps)
        
    SavedSteps = wNew
    #wHist = np.zeros(d)
    #MomentumHist = np.zeros(p)

        
        # Leapfrog steps
    for step in np.arange(RandomSteps):
            
            # Updating the Momentum
        dLdTheta = (X.T.dot(y - (1/(1 + np.exp(-f))).reshape(y.shape))) - (np.identity(d)*(1 / alpha)).dot(wNew)
        # The above line doesn't broad cast 5,1000 and (5,).  Is this on my end or what?
        trace1 = trInvGdG/2
            
            # Multiple fixed point iteration
        pm = p_momentum
        last_term = np.zeros(d)
        #MomentumHist = np.zeros((num_newton_steps,d))
        
        for fixed_i in range(num_newton_steps):
            #Momentum
            #MomentumHist[fixed_i,:] = pm
            #MomentumHist = np.r_[MomentumHist,pm]
            
            invGMomentum = invG.dot(pm.T)
            for dim in range(d):
                last_term[dim] = .5*(pm.dot(invGdG[dim].dot(invGMomentum)))
            
            pm = (p_momentum + TimeStep * (step_size / 2) * (dLdTheta.T - trace1 + last_term.T))
            print(pm)
            print("Multiple Fixed point iteration: ",fixed_i)

        print(p_momentum)
        # Update w parameters
        oInvGMomentum = la.inv(G).dot(p_momentum.T) # They use a matlab code \
        # This is either inv(G).dot(p_momentum)
              
        pw = wNew
        #wHist = np.zeros(d)
        for fixed_i in range(num_newton_steps):
            #wHist = np.r_[WHist,pw]
            
            f = X.dot(pw)
            p = 1 / (1 + np.exp(-f))
            
            v = p *(1 - p)
            G = (X.T * np.tile(v.T,(d,1))).dot(X) + (np.identity(d) / alpha)
            invGMomentum = la.inv(G).dot(p_momentum.T)
            print(pw)
            pw = (wNew.T + (Timestep * (step_size/2))*oInvGMomentum + (TimeStep*(step_size/2))*invGMomentum).T
            print(pw)
            print("Updating w parameters: ",fixed_i)
            
        wNew = pw
        
        # Calculate G based on new parameters
        f = X.dot(wNew)
        p = 1 / (1+ np.exp(-f))
        v = p *(1 - p)
        G = (X.T * np.tile(v.T,(d,1))).dot(X) + (np.identity(d) / alpha)
        invG = la.inv(G)
        
        # Calculate Partial derivatives for DG/dw
        for j in range(d):
            Z = ((1 - 2*p)*(X[:,j])) # Review
            g_deriv[j] = X.T.dot(v*(np.diag(Z))).dot(X) # My v is a scalar, but should it be?
            invGdG[j] = invG.dot(g_deriv[j])
            trInvGdG[j] = np.trace(invGdG[j])
            
            print("Partial derivatives for DG/dw: ",j)
            
        # Update Momentum
        invGMomentum = invG.dot(p_momentum.T)
        for j in range(d):
            last_term[j] = .5*(pm.dot(invGdG[j].dot(invGMomentum)))
            
        dHdTheta = - (X.T.dot(y - (p).reshape(y.shape))) - (np.identity(d)*(1/alpha)).dot(wNew) - (.5*trInvGdG.T + last_term.T).reshape(5,-1)
        #print(dHdTheta)
        p_momentum = p_momentum - TimeStep * (step_size/2) * dHdTheta.reshape(p_momentum.shape)
        #print(p_momentum)
        SavedSteps = np.r_[SavedSteps,wNew]
        
        print("leapfrog step: ",step)
        
    # Calculate propose H value
    log_prior = matlab_lognorm(np.zeros(d),wNew,alpha)
    f = X.dot(log_prior)
    loglike = f.T.dot(y) - np.sum(np.log(1 + np.exp(f)))
    propLJL = loglike + log_prior
    prop_log_det = .5 * (np.log(2) + d*np.log(np.pi) + 2 * np.sum(np.log(np.diagonal(la.cholesky(G)))))    
    propH = -propLJL + prop_log_det + (p_momentum.T.dot(invG.dot(p_momentum))) / 2
    
    # Calculate current Hamiltonian value
    current_log_det = .5 * (np.log(2) + d*np.log(np.pi) + 2 * np.sum(np.log(np.diagonal(la.cholesky(o_chol_G)))))    
    currentH = -currentLJL + current_log_det + (o_momentum.T.dot(o_inv_G).dot(o_momentum)) /2
    
    rat = -propH + currentH
    
    if (rat > 0 | (rat > np.log(np.random.rand(1)))):
        currentLJL = propLJL
        w = wNew
        acc = acc + 1
    if i > burnin:
        wSaved[i - burnin,:] = w
        
        
        
            
        
# Seems like things are working. 
# BUT! now the ratio is acting up because of an overload     

        
        
        

j=  4
[-158.13605443  249.31454384  264.29733539  415.44890287 -165.94499645]
[[-164.55159414  196.98867036  327.92118997  399.87580836 -118.05281957]]
Multiple Fixed point iteration:  0
[[-167.05211477  199.38287703  325.37920535  401.06691911 -119.31665672]]
Multiple Fixed point iteration:  1
[[-167.33990352  199.31551469  325.52869135  401.03695889 -118.92340684]]
Multiple Fixed point iteration:  2
[[-167.39250134  199.32414366  325.52647866  401.04288085 -118.90382616]]
Multiple Fixed point iteration:  3
[[-167.40004606  199.32471797  325.52742033  401.04343666 -118.89767293]]
Multiple Fixed point iteration:  4
[[-167.40128084  199.32483167  325.52749809  401.04352426 -118.89695774]]
Multiple Fixed point iteration:  5
[-158.13605443  249.31454384  264.29733539  415.44890287 -165.94499645]
[[0.001]
 [0.001]
 [0.001]
 [0.001]
 [0.001]]
[[ 0.31725915]
 [-0.4592973 ]
 [-0.52212514]
 [-1.07643964]
 [ 0.68212549]]
Updating w parameters:  0
[[ 0.31725915]
 [-0.4592973 ]
 [-0.52212514]
 [-

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:117: RuntimeWarning: overflow encountered in exp
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:70: RuntimeWarning: overflow encountered in exp
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:103: RuntimeWarning: overflow encountered in exp
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:147: RuntimeWarning: overflow encountered in exp
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:158: RuntimeWarning: invalid value encountered in greater


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [314]:
log_prior = np.sum(log_normal_pdf(np.zeros(d),wNew.reshape(1,-1),alpha))
log_normal_pdf(np.zeros(d).reshape(1,-1),wNew.reshape(1,-1),alpha)

matlab_lognorm(np.zeros(d),wNew.reshape(1,-1),alpha)
#lognorm.pdf(np.zeros(d),wNew.reshape(1,-1),alpha)
#f = X.dot(log_prior)
#loglike = f.T.dot(y) - np.sum(np.log(1 + np.exp(f)))
#propLJL = loglike + log_prior
#prop_log_det = .5 * (np.log(2) + d*np.log(np.pi) + 2 * np.sum(np.log(np.diagonal(la.cholesky(G)))))    
#propH = -propLJL + prop_log_det + (p_momentum.T.dot(invG.dot(p_momentum))) / 2
    
    # Calculate current Hamiltonian value
#current_log_det = .5 * (np.log(2) + d*np.log(np.pi) + 2 * np.sum(np.log(np.diagonal(la.cholesky(o_chol_G)))))    
#currentH = -currentLJL + current_log_det + (o_momentum.T.dot(o_inv_G).dot(o_momentum)) /2
    
#rat = -propH + currentH


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


array([[-1.05204146e+10, -7.37507198e+08, -7.55520478e+08,
        -3.25093586e+09, -3.49626345e+08]])

In [270]:
G = X.T.dot(v*(X)) + np.identity(d)/alpha

pw = wNew
wHist = np.zeros((num_newton_steps,d))
for fixed_i in range(num_newton_steps):
    wHist[fixed_i,:] = pw.T
            
    f = X.dot(pw)
    p = 1 / (1 + np.exp(-f))
    print(p[:10])        
    v = p *(1 - p)
    print(v[:10])
    G = (X.T * np.tile(v.T,(d,1))).dot(X) + (np.identity(d) / alpha)
    print(G)
    invGMomentum = la.inv(G).dot(p_momentum.T)
    print(la.inv(G))
    print(p_momentum)
        
    pw = wNew + (Timestep * (step_size/2))*oInvGMomentum + (TimeStep*(step_size/2))*invGMomentum
    print("Updating w parameters: ",fixed_i)


[[5.32863676e-04]
 [2.40145898e-02]
 [2.23726503e-04]
 [4.52068133e-01]
 [9.97699267e-01]
 [7.99353385e-01]
 [1.35018817e-01]
 [1.15944857e-01]
 [1.25076579e-01]
 [2.79385478e-01]]
[[5.32579732e-04]
 [2.34378893e-02]
 [2.23676450e-04]
 [2.47702536e-01]
 [2.29543990e-03]
 [1.60387551e-01]
 [1.16788736e-01]
 [1.02501647e-01]
 [1.09432428e-01]
 [2.01329233e-01]]
[[103.97869225   7.12323213  -7.87341002  -7.09848115 -20.74115732]
 [  7.12323213  93.70103357   1.5376068   -0.92358258  21.3662152 ]
 [ -7.87341002   1.5376068  100.12817323   0.48024362  -9.7142518 ]
 [ -7.09848115  -0.92358258   0.48024362 107.29518667  32.07131466]
 [-20.74115732  21.3662152   -9.7142518   32.07131466  41.05168398]]
[[ 0.01177931 -0.00317382  0.00193218 -0.0021736   0.00975865]
 [-0.00317382  0.01366032 -0.00161879  0.00343628 -0.01178099]
 [ 0.00193218 -0.00161879  0.0107164  -0.00161217  0.00561413]
 [-0.0021736   0.00343628 -0.00161217  0.01329447 -0.01365438]
 [ 0.00975865 -0.01178099  0.00561413 -0.0136

ValueError: array must not contain infs or NaNs

In [263]:
#G = (X.T * np.tile(v.T,(d,1))).dot(X) + (np.identity(d) / alpha)
print(X.T * np.tile(v.T,(d,1)))
print(wNew)
print(pw)

#f = X.dot(pw)
# pw = wNew + (Timestep * (step_size/2))*oInvGMomentum + (TimeStep*(step_size/2))*invGMomentum
print(wNew)
print(oInvGMomentum)
print(la.inv(G))
#la.inv(G).dot(p_momentum.T)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[-0.87646021]
 [ 1.1461239 ]
 [-0.56327717]
 [ 0.84712272]
 [-3.26939718]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[-0.87646021]
 [ 1.1461239 ]
 [-0.56327717]
 [ 0.84712272]
 [-3.26939718]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]]


ValueError: array must not contain infs or NaNs

In [254]:
#dHdTheta = - (X.T.dot(y - (p).reshape(y.shape))) - (np.identity(d)*(1/alpha)).dot(wNew) - (.5*trInvGdG.T + last_term.T).reshape(5,-1)      
#p_momentum = p_momentum - TimeStep * (step_size/2) * dHdTheta
# Solution here was changing the dHdTheta to matchthe same size
print(G)


[[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]


In [245]:
# Fix all the p_momentum stuff
# (p_momentum + TimeStep * (step_size / 2) * (dLdTheta.T - trace1 + last_term))
print(dLdTheta.T)
print(trace1)
print(last_term)
print(np.random.randn(d).dot(oG))

# Multiple fixed point iteration
#(p_momentum + TimeStep * (step_size / 2) * (dLdTheta.T - trace1 + last_term.T))
print(dLdTheta.T)


# UPdate Momeuntum
#p_momentum - TimeStep * (step_size/2) * dHdTheta
print(dHdTheta.T)

[[ 116.73857008  405.17478309 -162.57965502 -196.08716732 -177.39365391]]
[ 185.93963292  220.6418246    97.13920293 -227.79765778  -44.53418973]
[0. 0. 0. 0. 0.]
[ -51.17187522 -407.68768813  144.8176651   484.87852012   22.19823275]
[[ 116.73857008  405.17478309 -162.57965502 -196.08716732 -177.39365391]]
[[-22311783.24690811  -6318270.7692212   -2965026.82857529
    8134839.43424218   2301338.24694269]]


In [237]:
#dHdTheta = - (X.T.dot(y - (p).reshape(y.shape))) - (np.identity(d)*(1/alpha)).dot(wNew) - (.5*trInvGdG.T + last_term.T)     
#p_momentum = p_momentum - TimeStep * (step_size/2) * dHdTheta
print(dHdTheta) # This should be a 1x5 or 5x1.  SHould be 1x5
print((X.T.dot(y - (p).reshape(y.shape))).shape)
print(((np.identity(d)*(1/alpha)).dot(wNew)).shape )
print((.5*trInvGdG.T + last_term.T).shape )  

print(X.T.dot(y - (p).reshape(y.shape)) - ((np.identity(d)*(1/alpha)).dot(wNew)) - (.5*trInvGdG.T + last_term.T).reshape(d,-1))
pm

[[  71.0421173 ]
 [-183.61083143]
 [ 260.7202457 ]
 [ -31.47364913]
 [ 133.48536687]]
(5, 1)
(5, 1)
(5,)
[[ -70.00002   ]
 [ 184.24298118]
 [-259.85049841]
 [  32.06881138]
 [-132.78671264]]


array([[ -19.88202808,  -95.94197005, -222.96674368,   24.07274716,
         -29.6500946 ],
       [  43.7812091 ,  -32.27873287, -159.3035065 ,   87.73598435,
          34.01314258],
       [ -67.30156018, -143.36150216, -270.38627579,  -23.34678494,
         -77.0696267 ],
       [   5.74691353,  -70.31302845, -197.33780208,   49.70168877,
          -4.02115299],
       [ -35.49284047, -111.55278245, -238.57755608,    8.46193477,
         -45.26090699]])

In [223]:
#.5*(pm.T.dot(invGdG[j].dot(invGMomentum)))
print(pm)
print(invGdG[j])
print(invGMomentum)

[[253192.28492334  27694.49174741  18263.43046574 -61899.14083936
  -53456.60364939]
 [253134.84349015  27637.05031423  18205.98903255 -61956.58227254
  -53514.04508257]
 [253243.64551398  27745.85233805  18314.79105638 -61847.78024872
  -53405.24305875]
 [253222.39816944  27724.60499351  18293.54371184 -61869.02759326
  -53426.49040329]
 [253262.92033043  27765.12715451  18334.06587284 -61828.50543226
  -53385.96824229]]
[[-1.84438947e-01  1.36250750e-04  1.48022309e-02 -1.89985026e-02
   7.95209145e-03]
 [-6.45842003e-02  4.77103444e-05  5.18323410e-03 -6.65262470e-03
   2.78454998e-03]
 [ 1.51949037e-01 -1.12249449e-04 -1.21947385e-02  1.56518144e-02
  -6.55128787e-03]
 [ 4.80671406e-02 -3.55086820e-05 -3.85765003e-03  4.95125194e-03
  -2.07241639e-03]
 [-1.16828553e-01  8.63048624e-05  9.37612821e-03 -1.20341588e-02
   5.03706703e-03]]
[[ 30681.54089082]
 [ 16723.86762552]
 [ 22138.78015118]
 [-47320.22828099]
 [-61906.99203192]]


In [208]:
#(p_momentum + TimeStep * (step_size / 2) * (dLdTheta.T - trace1 + last_term.T)).T
#print(p_momentum)
print(dLdTheta.T)
print(trace1)
print(last_term.T)

print(np.random.randn(d).dot(oG))

[[ -97.18968098  343.85172478 -264.04783409   60.96992736 -107.98527056]]
[-27.1869078  159.59431956  -4.19707393  28.89907695  24.7993776 ]
[0. 0. 0. 0. 0.]
[140.27272868 -22.26727119 699.22085088 -87.988901     8.90756549]


In [217]:
#la.inv(G).dot(p_momentum)
print(G)
print(p_momentum)


[[ 2.50009652e+02 -1.42126482e-04 -9.93805785e-05 -1.82436575e-04
  -1.86189725e-04]
 [-1.42126482e-04  2.50009516e+02 -1.29955083e-01  6.99646317e+00
  -3.09522558e+00]
 [-9.93805785e-05 -1.29955083e-01  2.50009538e+02 -7.71982524e+00
  -1.42996062e+01]
 [-1.82436575e-04  6.99646317e+00 -7.71982524e+00  2.50009458e+02
   8.93047242e+01]
 [-1.86189725e-04 -3.09522558e+00 -1.42996062e+01  8.93047242e+01
   2.50009436e+02]]
[[-15.10761874  73.14687895 300.5044999  -40.84716125 167.41523642]]


In [182]:
#(X.T.dot(y - (1/(1 + np.exp(-f))).reshape(y.shape))) - (np.identity(d)*(1 / alpha)).dot(wNew)
print(X.T.dot(y - (1/(1 + np.exp(-f))).reshape(y.shape)))
print((np.identity(d)*(1 / alpha)).dot(wNew))
print((X.T.dot(y - (1/(1 + np.exp(-f))).reshape(y.shape))) - (np.identity(d)*(1 / alpha)).dot(wNew))
print(wNew)

[[ -70.24999968]
 [ 183.98922992]
 [-260.07832996]
 [  31.73025336]
 [-133.10859983]]
[[1.e-05]
 [1.e-05]
 [1.e-05]
 [1.e-05]
 [1.e-05]]
[[ -70.25000968]
 [ 183.98921992]
 [-260.07833996]
 [  31.73024336]
 [-133.10860983]]
[[0.001]
 [0.001]
 [0.001]
 [0.001]
 [0.001]]


In [185]:
print(p_momentum)

[-108.78881351 -541.84540847 -274.09872929  117.0023522    18.41367306]


In [170]:
#p_momentum + TimeStep * (step_size / 2) * (dLdTheta - trace1 + last_term.T)
print(p_momentum)
print(TimeStep)
print(step_size)
print(dLdTheta) 
print(trace1)
print(last_term.T)
print(wNew)

[-294330.11360976  203383.063632   -249447.47956989   11839.54781413
 -107200.16282618]
-1
0.5
[[-100.25000965 -100.25000965 -100.25000965 -100.25000965 -100.25000965]
 [ 172.74125941  172.74125941  172.74125941  172.74125941  172.74125941]
 [-261.78593999 -261.78593999 -261.78593999 -261.78593999 -261.78593999]
 [  40.6939741    40.6939741    40.6939741    40.6939741    40.6939741 ]
 [-141.72681417 -141.72681417 -141.72681417 -141.72681417 -141.72681417]]
[-0.00133021 -0.00178111 -0.00189239 -0.0025722  -0.00256598]
[-269.47578371  -48.57511033 -394.58686614 -266.6489555  -362.61929758]
[0.001 0.001 0.001 0.001 0.001]


In [167]:
(X.T.dot(y - (1/(1 + np.exp(-f))).reshape(y.shape))) - (np.identity(d)*(1 / alpha)).dot(wNew)

array([[-100.25000965, -100.25000965, -100.25000965, -100.25000965,
        -100.25000965],
       [ 172.74125941,  172.74125941,  172.74125941,  172.74125941,
         172.74125941],
       [-261.78593999, -261.78593999, -261.78593999, -261.78593999,
        -261.78593999],
       [  40.6939741 ,   40.6939741 ,   40.6939741 ,   40.6939741 ,
          40.6939741 ],
       [-141.72681417, -141.72681417, -141.72681417, -141.72681417,
        -141.72681417]])

In [ ]:
p_momentum - TimeStep * (step_size/2) * dHdTheta

In [155]:
MomentumHist

array([[-294330.11360976,  203383.063632  , -249447.47956989,
          11839.54781413, -107200.16282618],
       [      0.        ,       0.        ,       0.        ,
              0.        ,       0.        ],
       [      0.        ,       0.        ,       0.        ,
              0.        ,       0.        ],
       [      0.        ,       0.        ,       0.        ,
              0.        ,       0.        ],
       [      0.        ,       0.        ,       0.        ,
              0.        ,       0.        ],
       [      0.        ,       0.        ,       0.        ,
              0.        ,       0.        ]])

In [156]:
pm

array([[-294237.68249397,  203420.26946671, -249323.77082404,
          11931.27191236, -107084.44614087],
       [-294305.93031123,  203352.02164945, -249392.01864131,
          11863.0240951 , -107152.69395813],
       [-294197.29851138,  203460.6534493 , -249283.38684146,
          11971.65589495, -107044.06215828],
       [-294272.91848991,  203385.03347077, -249359.00681998,
          11896.03591642, -107119.68213681],
       [-294227.31329284,  203430.63866784, -249313.40162291,
          11941.64111349, -107074.07693974]])

In [152]:
print(X.T.shape)
print(y.shape)
print((1 /(1 + np.exp(-f))).reshape((1000,1)).shape)
print(y - ((1 /(1 + np.exp(-f))).reshape(y.shape)))
#(X.T.dot(y - (1/(1 + np.exp(-f))))) - (np.identity(d)*(1 / alpha)).dot(wNew)

(5, 1000)
(1000, 1)
(1000, 1)
[[ 0.50062253]
 [ 0.49869596]
 [-0.49995617]
 [-0.50009273]
 [ 0.49887146]
 [ 0.50012535]
 [-0.50142732]
 [ 0.49967573]
 [-0.50056263]
 [-0.50043123]
 [ 0.50056908]
 [-0.50125731]
 [ 0.50018836]
 [-0.50014874]
 [-0.50011638]
 [-0.50010392]
 [-0.50003985]
 [ 0.50039175]
 [ 0.50091055]
 [ 0.49987292]
 [ 0.49976586]
 [ 0.49962673]
 [-0.50102363]
 [ 0.50048291]
 [-0.50051636]
 [-0.50010613]
 [-0.50058603]
 [-0.49933846]
 [ 0.49988755]
 [-0.50024193]
 [-0.50151751]
 [-0.50142768]
 [ 0.49954752]
 [-0.50071709]
 [ 0.50040551]
 [ 0.49906475]
 [ 0.50091518]
 [-0.499885  ]
 [-0.50097717]
 [-0.50065046]
 [-0.50144119]
 [ 0.50001138]
 [ 0.49971811]
 [-0.50107865]
 [ 0.49928705]
 [-0.50110371]
 [-0.49982518]
 [-0.49958869]
 [-0.50028291]
 [-0.49917088]
 [ 0.49976926]
 [-0.50051289]
 [-0.4994852 ]
 [ 0.50121972]
 [-0.50074399]
 [ 0.49953896]
 [ 0.49976279]
 [-0.50056535]
 [-0.50033924]
 [-0.50020221]
 [-0.50026969]
 [-0.50141941]
 [-0.50007104]
 [ 0.49946336]
 [-0.50056

In [102]:
# Actual Analysis
type(v)
list(range(d))

test1 = np.array([1,2,3,4,5])
test2 = np.array([1,2,3,4,5])
test1 *test2

oG.shape
np.random.randn(d)
test = np.random.rand(1)
print(test)
print(leap_frog_steps)

np.ceil(test*leap_frog_steps).item()
np.arange(np.ceil(np.random.rand(1).item()*leap_frog_steps))


[0.2055339]
6


array([0., 1.])

In [123]:
#(X.T.dot(y - (1/(1 + np.exp(-f))))) - np.eye(d)*(1 / alpha).dot(wNew)
test_x = np.array([1,2,3,4,5])
np.tile(test_x,(5,1))
np.identity(5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [133]:
X = np.random.randn(10,4)
X = np.c_[np.ones(10),X]
y = np.random.binomial(1,.5,10)
beta = np.array([1,2,3,1,2])
print(y)
print(X.T.dot(y - 1 / (1 + np.exp(X.dot(beta)))))
(np.identity(d)*(1/alpha)).dot(beta)

(X.T.dot(y - (1/(1 + np.exp(-))))) - (np.identity(d)*(1 / alpha)).dot(wNew)

[1 0 0 0 0 1 1 1 0 0]
[-0.46309475  1.46624735  3.41673387  0.18658158 -0.63670486]


ValueError: operands could not be broadcast together with shapes (10,) (1000,) 

In [324]:
test1 = np.array([1,2,3,4,5])
test2 = np.array([1,2,3,2,1]).reshape(-1,1)
np.c_[test1,test2]

array([[1, 1],
       [2, 2],
       [3, 3],
       [4, 2],
       [5, 1]])